# Quarterly growth screen (EPS + revenue)

Use the JSON exports in `output/*-facts-json` to find tickers where:
- EPS is near/just above zero (window controlled below).
- Revenue/EPS can optionally be required to rise quarter-over-quarter on the latest consecutive periods.

Adjust the config cell for thresholds. No widget UI is included; edit the variables directly, rerun the helper cell, then rerun the results cell.


In [91]:
from pathlib import Path
import json

DATA_DIR = Path("output")
RECENT_PERIODS = 4 # how many latest overlapping quarters to check (must be consecutive)

# EPS window to count as "about to go positive"
eps_lower, eps_upper = -0.2, 0.4
#eps_lower, eps_upper = -100, 100

# Toggle whether each metric must grow quarter-over-quarter
require_eps_growth = True
require_rev_growth = True

# Minimum per-quarter step (if growth is required). Use negatives to allow mild declines.
eps_growth_min = 0.0
rev_growth_min = 0.0


In [92]:
QUARTER_ORDER = {"q1": 1, "q2": 2, "q3": 3, "q4": 4}


def load_metric_series(ticker: str, prefix: str, base_dir: Path = DATA_DIR):
    """Return a dict mapping (year, quarter) -> value for the given metric."""
    path = base_dir / f"{ticker}-facts-json" / f"{prefix}_{ticker}.json"
    if not path.exists():
        return None
    data = json.load(path.open())
    years = data.get("years")
    if not years:
        return None
    series = {}
    for year, quarters in years.items():
        for quarter, value in quarters.items():
            q_key = quarter.lower()
            if q_key not in QUARTER_ORDER:
                continue
            series[(int(year), QUARTER_ORDER[q_key])] = float(value)
    return series or None


def sort_periods(periods):
    return sorted(periods, key=lambda p: (p[0], p[1]))


def consecutive(periods):
    seq = [year * 4 + quarter for year, quarter in periods]
    return all(b - a == 1 for a, b in zip(seq, seq[1:]))


def is_increasing(values, tolerance):
    return all((b - a) > tolerance for a, b in zip(values, values[1:]))


def meets_growth(values, min_step, required):
    if not required:
        return True
    return is_increasing(values, tolerance=min_step)


def period_label(period):
    year, quarter = period
    return f"{year}Q{quarter}"


def screen_ticker(ticker: str):
    eps = load_metric_series(ticker, "epsd")
    rev = load_metric_series(ticker, "rev")
    if not eps or not rev:
        return None

    common_periods = sort_periods(set(eps) & set(rev))
    if len(common_periods) < 3:
        return None

    recent_periods = common_periods[-RECENT_PERIODS:]
    if len(recent_periods) < 3 or not consecutive(recent_periods):
        return None

    eps_values = [eps[p] for p in recent_periods]
    rev_values = [rev[p] for p in recent_periods]
    eps_in_window = all(
        eps_lower <= value <= eps_upper for value in eps_values
    )

    checks = [
        meets_growth(eps_values, eps_growth_min, require_eps_growth),
        meets_growth(rev_values, rev_growth_min, require_rev_growth),
        eps_in_window,
        rev_values[-1] > 0,
    ]
    if not all(checks):
        return None

    return {
        "ticker": ticker.upper(),
        "periods": [period_label(p) for p in recent_periods],
        "eps_values": eps_values,
        "revenue_values": rev_values,
        "eps_last": eps_values[-1],
    }


In [93]:
tickers = sorted(
    d.name.replace("-facts-json", "")
    for d in DATA_DIR.iterdir()
    if d.is_dir() and d.name.endswith("-facts-json")
)

results = []
for ticker in tickers:
    screened = screen_ticker(ticker)
    if screened:
        results.append(screened)

print(f"Scanned {len(tickers)} tickers")
print(f"Matches with current settings: {len(results)}")

try:
    import pandas as pd  # optional
except ImportError:
    pd = None

if pd:
    display(pd.DataFrame(results).sort_values("eps_last"))
else:
    for entry in results:
        print("Ticker:", entry["ticker"])
        print("Periods:", ", ".join(entry["periods"]))
        print("EPS trend:", entry["eps_values"])
        print("Revenue trend:", entry["revenue_values"])


Scanned 8667 tickers
Matches with current settings: 14


,ticker,periods,eps_values,revenue_values,eps_last
0,AMPX,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[-0.11, -0.08, -0.05, -0.03]","[12884000.0, 13645000.0, 15067000.0, 21426000.0]",-0.03
1,AMPX-WT,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[-0.11, -0.08, -0.05, -0.03]","[12884000.0, 13645000.0, 15067000.0, 21426000.0]",-0.03
8,LXU,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[-0.13, -0.02, 0.04, 0.1]","[126961000.0, 143432000.0, 151296000.0, 155431...",0.10
6,HL-PB,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[0.03, 0.05, 0.09, 0.15]","[249655000.0, 261339000.0, 304027000.0, 409542...",0.15
5,HL,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[0.03, 0.05, 0.09, 0.15]","[249655000.0, 261339000.0, 304027000.0, 409542...",0.15
10,PLTR,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[0.03, 0.08, 0.13, 0.18]","[827519000.0, 883855000.0, 1003697000.0, 11810...",0.18
12,QTWO,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[0.01, 0.07, 0.18, 0.23]","[183045000.0, 189735000.0, 195148000.0, 201704...",0.23
9,OSW,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[0.13, 0.15, 0.19, 0.23]","[217206000.0, 219630000.0, 240726000.0, 258518...",0.23
4,HDSN,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[-0.05, 0.06, 0.23, 0.27]","[34643000.0, 55343000.0, 72849000.0, 74012000.0]",0.27
11,PRLB,"[2024Q4, 2025Q1, 2025Q2, 2025Q3]","[-0.01, 0.15, 0.18, 0.3]","[121750000.0, 126205000.0, 135063000.0, 135366...",0.30
